# Getting Embeddings

In [ ]:
from transformers import BertModel, BertTokenizer,BertForPreTraining, BertConfig
from tqdm.auto import tqdm
import torch
import nltk
from pathlib import Path
import re
from scipy.spatial.distance import  cosine
import pickle
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
DATA_DIR = Path('data')
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
SEEDS = ['finance', 'medicine', 'sports', 'technology']
NUM_WORDS_PER_SET = 10

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")


def get_embeddings(model, tokens, embedding_size=768):
    with torch.no_grad():
        output = model(**tokens)
        embedding = output.last_hidden_state[0][1]
        return torch.reshape(embedding, (embedding_size, ))


In [34]:
def get_embeddings_batch(model, tokens, embedding_size=768, batch_size=4):
    with torch.no_grad():
        output = model(**tokens)
        embedding = output.last_hidden_state[:, 1, :]
        return embedding


def cos_distance_batch(topic, words):
    return np.inner(
        topic, words) / (np.linalg.norm(topic) * np.linalg.norm(words, axis=1))


def job(vocab, topic, tokenizer, model, batch_size=4):
    res_col = np.zeros((len(vocab), ))
    vocab = list(vocab)
    loop = tqdm(range(0, len(vocab), batch_size))
    loop.set_description(f"topic: {topic}")
    topic_token = tokenizer(topic,
                            return_tensors='pt',
                            padding=True,
                            max_length=10,
                            truncation=True)
    topic_emb = get_embeddings(model, topic_token)
    for batch_index in loop:
        lo = batch_index
        hi = min(batch_index + batch_size, len(vocab))
        batch = vocab[batch_index:batch_index + batch_size]
        tokens = tokenizer(batch,
                           return_tensors='pt',
                           padding='max_length',
                           max_length=10,
                           truncation=True)
        # if len(token['input_ids']) > 3:
        #     print(f"WARNING: Word '{word}' is not in BERT's vocabulary")
        word_embs = get_embeddings_batch(model, tokens)
        res_col[lo:hi] = cos_distance_batch(topic_emb, word_embs)
        # res_col.append(cosine(topic_emb, word_emb))
    return res_col

In [ ]:
token = tokenizer("medicine", return_tensors='pt', truncation=True)
print(token.input_ids.shape)
out = model(**token)
out.last_hidden_state.shape

In [ ]:

tokens = {
    "medicine": tokenizer("medicine", return_tensors='pt', truncation=True),
    "cat": tokenizer("cat", return_tensors='pt', truncation=True),
    "kitty": tokenizer('kitty', return_tensors='pt', truncation=True),
    "feline": tokenizer('feline', return_tensors='pt', truncation=True),
    "doctor": tokenizer("doctor", return_tensors='pt', truncation=True),
    "medical": tokenizer("medical", return_tensors='pt', truncation=True),
}
embeddings = {}
for key, val in tokens.items():
    embeddings[key] = get_embeddings(model, val)

print("cat-kitty: " + f"{cosine(embeddings['cat'], embeddings['kitty'])}")
print('cat-doctor: ' + f"{cosine(embeddings['cat'], embeddings['doctor'])}")
print('medicine-cat: ' +
      f"{cosine(embeddings['medicine'], embeddings['cat'])}")
print('medicine-medical: ' +
      f"{cosine(embeddings['medicine'], embeddings['medical'])}")


In [ ]:
tokenizer('[MASK]')

In [ ]:
cat_emb = get_embeddings(cat_outputs)
hello_emb = get_embeddings(hello_outputs)
hi_emb = get_embeddings(hi_outputs)

print(cosine(hi_emb, hello_emb))
print(cosine(hi_emb, cat_emb))

In [ ]:
cosine([1], [0.1])

# Building vocab for the corpus

In [ ]:
df = pd.read_csv(Path('data') / 'dataset.csv')
df.head()

In [ ]:
try:
    nltk.data.find('tokenizers/punkt.zip')
except LookupError:
    nltk.download('punkt')

try:
    nltk.data.find('corpora/stopwords.zip')
except:
    nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
stop_words = nltk.corpus.stopwords.words('english')

In [25]:
def vocab_preprocess(row, lemmatize=True):
    from nltk.stem import WordNetLemmatizer
    row = row.lower()
    row = row.split('-')[1::]
    row = ''.join(row)
    lemmatizer = WordNetLemmatizer()
    # tokenize words
    words = re.findall(re.compile('[a-zA-Z]+'), row)
    # Remove stop words
    words = [word for word in words if word not in stop_words]
    if (lemmatize):
        words = [lemmatizer.lemmatize(word) for word in words]

    return words

In [27]:
df = pd.read_csv(DATA_DIR / 'dataset.csv')
desc = df['description'].astype(str)
data = []
for i, row in desc.iteritems():
    data.append(vocab_preprocess(row, False))
vocab = set()
for words in data:
    vocab = vocab.union(set(words))
with open(DATA_DIR / "vocab" / "global_vocab_no_lemmatize.pkl", "wb") as f:
    pickle.dump(vocab, f)

# Build global rankings

In [45]:
vocab = None
with open(DATA_DIR / "vocab" / "global_vocab_no_lemmatize.pkl", "rb") as f:
    vocab = pickle.load(f)
print(len(vocab))

50139


In [46]:
'pathology' in vocab

True

In [ ]:

topic_embeddings = []
cols = []
cos
for seed in SEEDS:
    cols.append(job(vocab, seed, tokenizer, model, batch_size=128))


In [ ]:
arr = np.array(cols)
res_dict = {
    '_vocab': list(vocab),
}
for i, topic in enumerate(SEEDS):
    print(arr[i].shape)
    res_dict[topic] = arr[i]
res_df = pd.DataFrame(res_dict)
res_df = res_df.set_index(['_vocab'])
res_df.to_csv(DATA_DIR / 'global_cos_similarity.csv')
res_df.sort_values('finance', ascending=False).head(10)

In [ ]:
df = pd.read_csv('./global_cos_similarity.csv', index_col='_vocab')
for seed in SEEDS:
    line = df.sort_values(seed, ascending=False).head(4).index[1::]
    print(" ".join(line))

In [ ]:
res_df.sort_values(by='finance', ascending=False).head(10)

# Initialize word sets from $e$

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
res_df = pd.read_csv('global_cos_similarity.csv')
res_df = res_df.set_index(['_vocab'])
word_set = {}
added_words = set()
for seed in SEEDS:
    word_set[seed] = []
    col = res_df[seed]
    sorted_col = col.sort_values()[::-1]
    i = 0
    for word, _ in sorted_col.iteritems():
        if i == NUM_WORDS_PER_SET + 1:
            break
        if word not in added_words:
            word_set[seed].append(word)
            added_words.add(word)
            i += 1

pd.DataFrame(word_set)

accounting logistics marketing banking procurement financial debt engineering advertising management
pharmacy pathology medical biology surgery health accounting astronomy logistics industry
baseball basketball athletics gymnastics volleyball football swimming hockey education broadcasting
engineering robotics telecommunication communication logistics journalism industry philosophy industrial marketing


# Local Knowlege using pretrained BERT

In [ ]:
# Prepare datasets
from transformers import BertTokenizer, DataCollatorForLanguageModeling
from datasets import load_dataset, concatenate_datasets
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15,
)
dataset = load_dataset('ag_news')

In [ ]:
concatenate_datasets([dataset['train'], dataset['test']], axis=0)

In [ ]:
%%bash

python3 scripts/train.py \
    -o ./models \
    -t ./data/tokens/tokens-pretrained-30522.pkl \
    -n bert-pretrined-30522 

# Compute Local Cosine similarity

In [ ]:
model = BertModel.from_pretrained('models/bert-pretrained-pretrained')
model.eval()
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
a = np.ones((768, ))
b = np.random.rand(4, 768)
np_cos = np.inner(a, b) / (np.linalg.norm(a) * np.linalg.norm(b, axis=1))
scp_cos = [1-cosine(a, b[i]) for i in range(4)]
print(np_cos.shape)
print(scp_cos)

In [4]:
with open('vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)

In [49]:
def get_embeddings_batch(model, tokens, embedding_size=768, batch_size=4):
    with torch.no_grad():
        output = model(**tokens)
        embedding = output.last_hidden_state[:, 1, :]
        return embedding


def cos_distance_batch(topic, words):
    return np.inner(topic, words) / (np.linalg.norm(topic) * np.linalg.norm(words, axis=1))


def job(vocab, topic, tokenizer, model, batch_size=4):
    res_col = np.zeros((len(vocab), ))
    vocab = list(vocab)
    loop = tqdm(range(0, len(vocab), batch_size))
    loop.set_description(f"topic: {topic}")
    topic_token = tokenizer(topic, return_tensors='pt', padding=True, max_length=10, truncation=True)
    topic_emb = get_embeddings(model, topic_token)
    for batch_index in loop:
        lo = batch_index
        hi = min(batch_index + batch_size, len(vocab))
        batch = vocab[batch_index:batch_index + batch_size]
        tokens = tokenizer(batch, return_tensors='pt', padding='max_length', max_length=10, truncation=True)
        # if len(token['input_ids']) > 3:
        #     print(f"WARNING: Word '{word}' is not in BERT's vocabulary")
        word_embs = get_embeddings_batch(model, tokens)
        res_col[lo:hi] = cos_distance_batch(topic_emb, word_embs)
        # res_col.append(cosine(topic_emb, word_emb))
    return res_col

In [ ]:
res = []
for topic in SEEDS:
    res_col = job(vocab, topic, tokenizer, model, batch_size=256)
    res.append(res_col)

In [51]:

arr = np.array(res)
arr = arr.T
res_dict = {    
    '_vocab': list(vocab),
}
for i, topic in enumerate(SEEDS):
    res_dict[topic] = arr[:, i]
res_df = pd.DataFrame(res_dict)
res_df = res_df.set_index(['_vocab'])
res_df.to_csv('local_embeddings_bert-pretrained-pretrained.csv')

In [ ]:
res_df = pd.read_csv('./local_embeddings_bert-pretrained-pretrained.csv', index_col='_vocab')
res_df.sort_values('finance', ascending=False).head(10)

In [32]:
# BERTopic

In [ ]:
from bertopic import BERTopic

seeds = np.array([SEEDS]).T.tolist()
topic_model = BERTopic(seed_topic_list=seeds)
docs = pd.read_csv(DATA_DIR / 'dataset.csv')['description']
topic_model.fit_transform(docs)

In [ ]:
similar_topics, similarity = topic_model.find_topics("finance", top_n=5)
for i, topic in enumerate(similar_topics):
    print(topic_model.get_topic(topic)[0][0], similarity[i])

# Cate 